In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train_data=pd.read_csv('/Users/s0c02nj/Desktop/WNS/Old_versions/train_LZdllcl.csv')
test_data=pd.read_csv('/Users/s0c02nj/Desktop/WNS/Old_versions/test_2umaH9m.csv')
test_x=test_data
train_x=train_data.drop(columns=['is_promoted'])#-----removing response

In [4]:
#len(train_data)
#len(test_data)
len(train_data)

54808

In [5]:
#COMBO contains combined doc for preprocessings
train_x=train_data.drop(columns=['is_promoted'])#-----removing response


In [6]:
combo=pd.concat([train_x,test_x],axis=0,sort=False)
combo.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73


In [ ]:
#train_data.head()

In [7]:
#train_data.head()
#combo.tail()
#len(combo)
#combo.info()
len(train_data)+len(test_data)

78298

In [8]:
#Missing Values
print combo['previous_year_rating'].mode()[0]
combo['education'].mode()[0]

3.0


"Bachelor's"

In [ ]:
#combo['previous_year_rating'].head()

In [9]:
#Replacing Missing with max frequency
combo['previous_year_rating']=combo['previous_year_rating'].fillna(3)
combo['education']=combo['education'].fillna("Bachelor's")

In [ ]:
#combo.isnull().sum()

In [10]:
combo['previous_year_rating'].head()

0    5.0
1    5.0
2    3.0
3    1.0
4    3.0
Name: previous_year_rating, dtype: float64

In [ ]:
#employee_id
#department ----- Categorical
#region ---- Categorical 
#education --- Categorical
#gender --- Categorical
#recruitment_channel-- Categorical
#no_of_trainings ----> Cont 
#age-----------------> Cont
#previous_year_rating-- categorical
#length_of_service ---- Cont
#KPIs_met >80% --- Categorical
#awards_won ----- Categorical
#avg_training_score --- Cont

In [11]:
#Modifying the datatypes---- 
combo['department'] = combo.department.astype('category')
combo['region'] = combo.region.astype('category')
combo['education'] = combo.education.astype('category')
combo['gender'] = combo.gender.astype('category')
combo['recruitment_channel'] = combo.recruitment_channel.astype('category')
#combo['previous_year_rating'] = combo.previous_year_rating.astype('category')
combo['KPIs_met >80%'] = combo['KPIs_met >80%'].astype('category')
combo['awards_won?'] = combo['awards_won?'].astype('category')

In [12]:
#Filter
data_categorical = combo.filter(['department','region','education','gender','recruitment_channel'], axis=1)
data_categorical.head()


,department,region,education,gender,recruitment_channel
0,Sales & Marketing,region_7,Master's & above,f,sourcing
1,Operations,region_22,Bachelor's,m,other
2,Sales & Marketing,region_19,Bachelor's,m,sourcing
3,Sales & Marketing,region_23,Bachelor's,m,other
4,Technology,region_26,Bachelor's,m,other


In [15]:
#Label Encode
from sklearn.preprocessing import LabelEncoder
categorical_encoded=data_categorical.apply(LabelEncoder().fit_transform)

In [16]:
categorical_encoded['previous_year_rating']=combo['previous_year_rating']
categorical_encoded['KPIs_met']=combo['KPIs_met >80%']
categorical_encoded['awards_won']=combo['awards_won?']
categorical_encoded.dtypes

department                 int64
region                     int64
education                  int64
gender                     int64
recruitment_channel        int64
previous_year_rating     float64
KPIs_met                category
awards_won              category
dtype: object

In [17]:
#Modifying the datatypes
categorical_encoded['department'] = categorical_encoded.department.astype('category')
categorical_encoded['region'] = categorical_encoded.region.astype('category')
categorical_encoded['education'] = categorical_encoded.education.astype('category')
categorical_encoded['gender'] = categorical_encoded.gender.astype('category')
categorical_encoded['recruitment_channel'] = categorical_encoded.recruitment_channel.astype('category')
categorical_encoded['no_of_trainings']=combo['no_of_trainings']
categorical_encoded['age']=combo['age']
categorical_encoded['length_of_service']=combo['length_of_service']
categorical_encoded['avg_training_score']=combo['avg_training_score']
#categorical_encoded['employee_id']=combo['employee_id']
#cat_encoded.dtypes

In [18]:
X_train=categorical_encoded[0:54808]
X_test=categorical_encoded[54808:78298]
Y_train=train_data.is_promoted

In [21]:
from sklearn.model_selection import train_test_split
x1_train, x1_val, y1_train, y1_val = train_test_split(X_train, Y_train,test_size = .2,random_state=11)

In [31]:
#Model
from imblearn.ensemble import BalancedBaggingClassifier
im_model= BalancedBaggingClassifier(base_estimator=None, bootstrap=True,
             bootstrap_features=True, max_features=6, max_samples=1.0,
             n_estimators=500, n_jobs=2, oob_score=True, random_state=0,
             ratio='auto', replacement=False, verbose=0, warm_start=False)


im_model.fit(x1_train,y1_train)


BalancedBaggingClassifier(base_estimator=None, bootstrap=True,
             bootstrap_features=True, max_features=6, max_samples=1.0,
             n_estimators=500, n_jobs=2, oob_score=True, random_state=0,
             ratio='auto', replacement=False, verbose=0, warm_start=False)

In [33]:
im_model.score(x1_val, y1_val)
print recall_score(y1_val, im_model.predict(x1_val))
print precision_score(y1_val, im_model.predict(x1_val))
f1_score(y1_val, im_model.predict(x1_val))

0.6570545829042225
0.27335047129391604


0.38608169440242057

In [ ]:
#sm = SMOTE(random_state=12, ratio = 1)
#x1_train_res, y1_train_res = sm.fit_sample(x1_train, y1_train)

In [ ]:
#Random Forest

In [ ]:
#clf_rf = RandomForestClassifier(n_estimators=500, random_state=12,criterion='gini',max_depth=None)
clf_rf=RandomForestClassifier(random_state=0,max_depth=20,min_samples_split=200,
                              min_samples_leaf=10,criterion='gini',n_estimators=500)

clf_rf.fit(x1_train, y1_train)

In [ ]:
clf_rf.score(x1_val, y1_val)
recall_score(y1_val, clf_rf.predict(x1_val))
precision_score(y1_val, clf_rf.predict(x1_val))
f1_score(y1_val, clf_rf.predict(x1_val))

In [ ]:
np.true_divide(np.count_nonzero(Y_train==0),len(Y_train))

In [ ]:
clf_rf.feature_importances_

# GBM1

In [4]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier as gbm
gbm_model=gbm(loss='deviance', learning_rate=0.01, n_estimators=600, subsample=1.0, 
              criterion='friedman_mse', min_samples_split=900, min_samples_leaf=10, 
              min_weight_fraction_leaf=0.0, max_depth=15,max_features='sqrt',random_state=12)

In [3]:
gbm()

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [ ]:
gbm_model.fit(X_train, Y_train)
#X_train

In [ ]:
#gbm_model.fit(x1_train, y1_train)

In [ ]:
gbm_model.score(x1_val, y1_val)
print recall_score(y1_val, gbm_model.predict(x1_val))
print precision_score(y1_val, gbm_model.predict(x1_val))
f1_score(y1_val, gbm_model.predict(x1_val))

In [ ]:
#Threshold determination

In [ ]:
#predict = gbm_model.predict(x1_val)
from sklearn.metrics import roc_auc_score
predict_probabilities_gbm = gbm_model.predict_proba(X_train)[:,1]
#roc_auc_score(y1_val, predict_probabilities)


In [ ]:
predict_mine_gbm = np.where(predict_probabilities > 0.314, 1, 0)
print recall_score(y1_val, predict_mine_gbm)
print precision_score(y1_val, predict_mine_gbm)
print f1_score(y1_val, predict_mine_gbm)

In [ ]:
predict_probabilities_gbm_test= gbm_model.predict_proba(X_test)[:,1]


In [ ]:
final_pred_gbm=np.where(predict_probabilities_gbm_test > 0.3149, 1, 0)

In [ ]:
#prop_1=np.true_divide(np.count_nonzero(y1_train==1),len(y1_train))
#prop_0=np.true_divide(np.count_nonzero(y1_train==0),len(y1_train))
#prop_1
np.savetxt("/Users/s0c02nj/Desktop/WNS/probability_gbm.csv",predict_probabilities_gbm_test)

In [ ]:
#Gaussian

In [ ]:
#XGBOOST

In [39]:
xgb_model = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 30,
 min_child_weight= 2,
 learning_rate=0.01,
 gamma=0.9,                        
 subsample=0.9,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)

In [36]:
xgb.XGBClassifier()
xgb_model.fit(np.array(x1_train), np.array(y1_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.9, learning_rate=0.01,
       max_delta_step=0, max_depth=30, min_child_weight=2, missing=None,
       n_estimators=1000, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.9)

In [37]:
xgb_model.score(np.array(x1_val), np.array(y1_val))

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9374201787994891

In [38]:
cross_val_score(xgb_model,np.array(X_train),np.array(Y_train),cv=5,scoring='f1')

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty ar

array([0.50489827, 0.48317128, 0.49737828, 0.52670623, 0.5       ])

In [43]:
emp_ids=train_data.employee_id
len(emp_ids)

54808

In [45]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, random_state=2, shuffle=True)


final_emp_ids = []
final_ans = []


oof_predictions = []
i = 0 

for train_index, test_index in kf.split(X_train, Y_train):
    
    #path = "wns_fold_"+str(i)+".hdf5"
    #checkpoint = ModelCheckpoint(filepath=path, verbose=1, 
    #                             monitor='val_f1', 
    #                             save_best_only=True, mode='max')
    
    x_t = np.array(X_train.iloc[train_index])
    y_t = np.array(Y_train[train_index])

    x_v = np.array(X_train.iloc[test_index])
    y_v = np.array(Y_train[test_index])

    #x_t = get_data(x_t)
    #x_v = get_data(x_v)
    
    final_emp_ids.append(emp_ids[test_index])
    
    xgb_model = xgb.XGBClassifier(n_estimators= 2000,max_depth= 30,min_child_weight= 2,learning_rate=0.01,gamma=0.9,subsample=0.9,colsample_bytree=0.8,objective= 'binary:logistic',nthread= -1,scale_pos_weight=1)
    
    xgb_model.fit(x_t, y_t)

    #model.load_weights(path)

    #x_te = get_data(x_test)
    probs = xgb_model.predict_proba(np.array(X_test))[:,1]
    oof_predictions.append(xgb_model.predict_proba(x_v)[:,1])
    final_ans.append(probs)
    
    i += 1

In [41]:
X_train.shape
X_test.shape

(23490, 12)

In [46]:
flat_oof = [pred for sublist in oof_predictions for pred in sublist]

In [47]:
final_probs = np.average(final_ans, axis=0)

In [48]:
flat_final_emp_ids = [pred for sublist in final_emp_ids for pred in sublist]

In [51]:
np.savetxt("/Users/s0c02nj/Desktop/WNS/probability_xgboost_avg_test_pred.csv",final_probs)
np.savetxt("/Users/s0c02nj/Desktop/WNS/probability_xgboost_oof.csv",flat_oof)
np.savetxt("/Users/s0c02nj/Desktop/WNS/probability_xgboost_empids.csv",flat_final_emp_ids)


In [50]:
np.shape(final_probs)

(23490,)

In [52]:
train_data.employee_id

0        65438
1        65141
2         7513
3         2542
4        48945
5        58896
6        20379
7        16290
8        73202
9        28911
10       29934
11       49017
12       60051
13       38401
14       77040
15       43931
16        7152
17        9403
18       17436
19       54461
20       12067
21       33332
22       58789
23       71177
24       52057
25       26585
26       37749
27        2370
28        8219
29       74759
         ...  
54778    30691
54779    49923
54780    73767
54781    76497
54782    17281
54783    45504
54784    68370
54785    42055
54786    32291
54787    77455
54788    75164
54789    75942
54790    67988
54791    27274
54792      994
54793    28400
54794    35000
54795    13477
54796    12592
54797    62450
54798    40257
54799    68093
54800    39227
54801    12431
54802     6915
54803     3030
54804    74592
54805    13918
54806    13614
54807    51526
Name: employee_id, Length: 54808, dtype: int64

# FINAL PRED

In [60]:
mydata=pd.read_csv('/Users/s0c02nj/Downloads/oof_df_final.csv')
mydata.head()
train_x=mydata.drop(['is_promoted'],axis=1)

train_x.head()
train_y=mydata.is_promoted

In [62]:
train_y.head()

0    0
1    0
2    0
3    0
4    0
Name: is_promoted, dtype: int64

In [68]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier as gbm
xgb_model = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 30,
 min_child_weight= 2,
 learning_rate=0.01,
 gamma=0.9,                        
 subsample=0.9,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1,random_state=1)

In [69]:
xgb_model.fit(train_x,train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.9, learning_rate=0.01,
       max_delta_step=0, max_depth=30, min_child_weight=2, missing=None,
       n_estimators=2000, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.9)

In [70]:
xgb_model.score(train_x,train_y)

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9945445920303605

In [71]:
cross_val_score(xgb_model,train_x,train_y,cv=5,scoring='f1')

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty ar

array([0.50873197, 0.4928839 , 0.52592036, 0.53393665, 0.50339111])

In [72]:
#predict_mine_gbm = np.where(predict_probabilities_xgb > 0.38, 1, 0)
#print recall_score(np.array(Y_train), predict_mine_xgb)
#print precision_score(np.array(Y_train), predict_mine_xgb)
#print f1_score(np.array(Y_train), predict_mine_xgb)

In [76]:
test_x.head()
train_x.head()

,oof_dense1,oof_dense2,oof_dense3,oof_gbm,oof_xgboost
0,0.195576,0.309827,0.255652,0.26400,0.123000
1,0.000171,0.000043,0.000021,0.00890,0.001140
2,0.004140,0.001743,0.000830,0.00558,0.000729
3,0.109427,0.129724,0.111394,0.09370,0.034900
4,0.007434,0.004312,0.002423,0.03100,0.003190


In [77]:
test_data=pd.read_csv('/Users/s0c02nj/Downloads/for_oof_test_predictions.csv')
test_x=test_data.drop(['employee_id'],axis=1)

In [78]:
pred_prob_xgb= xgb_model.predict_proba(test_x)[:,1]

In [79]:
np.savetxt("/Users/s0c02nj/Desktop/WNS/model_xgb_win.csv",pred_prob_xgb)

In [22]:
print recall_score(np.array(Y_train), xgb_model.predict(np.array(X_train)))
print precision_score(np.array(Y_train), xgb_model.predict(np.array(X_train)))
#f1_score(y1_val, gbm_model.predict(X_train))

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8847179487179487
0.9983796296296297


/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
f1_score(np.array(Y_train), xgb_model.predict(np.array(X_train)))

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9381185426862424

In [ ]:
prediction_xgb_final=xgb_model.predict(np.array(X_test))
#pre

In [ ]:
np.savetxt("/Users/s0c02nj/Desktop/WNS/model_xgb.csv",prediction_xgb)

In [24]:
predict_probabilities_xgb = xgb_model.predict_proba(np.array(X_train))[:,1]

In [ ]:
#predict_probabilities_xgb=predict_probabilities

In [41]:
predict_mine_xgb = np.where(predict_probabilities_xgb > 0.38, 1, 0)

In [42]:
print recall_score(np.array(Y_train), predict_mine_xgb)
print precision_score(np.array(Y_train), predict_mine_xgb)
print f1_score(np.array(Y_train), predict_mine_xgb)

0.9653333333333334
0.990945462202569
0.9779717373233584


In [50]:
predict_probabilities_xgb = xgb_model.predict_proba(np.array(X_test))

In [51]:
predict_probabilities_xgb = predict_probabilities_xgb[:,1]

In [52]:
np.savetxt("/Users/s0c02nj/Desktop/WNS/probability_xgb_final.csv",predict_probabilities_xgb)

# GBM2

In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier as gbm
gbm_model=gbm(loss='deviance', learning_rate=0.01, n_estimators=300, subsample=1.0, 
              criterion='friedman_mse', min_samples_split=400, min_samples_leaf=15, 
              min_weight_fraction_leaf=0.0, max_depth=40,max_features='sqrt',random_state=12)

In [ ]:
gbm_model.fit(x1_train_res, y1_train_res)

In [ ]:
gbm_model.score(x1_val, y1_val)
recall_score(y1_val, gbm_model.predict(x1_val))
#precision_score(y1_val, gbm_model.predict(x1_val))
#f1_score(y1_val, gbm_model.predict(x1_val))

In [ ]:
from sklearn.grid_search import GridSearchCV 

In [ ]:
param_test2 = {'max_depth':range(5,20,2), 'min_samples_split':range(200,1001,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.01, n_estimators=500, max_features='sqrt', subsample=1, random_state=10), 
param_grid = param_test2, scoring='f1',n_jobs=7,iid=False, cv=10)
gsearch2.fit(x1_train_res,y1_train_res)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_





In [ ]:
len(x1_train_res)

In [ ]:
#gbm.score(x1_val, y1_val)
#recall_score(y1_val, gbm.predict(x1_val))
#precision_score(y_val, gbm.predict(x_val))
#f1_score(y1_val, gbm.predict(x1_val))
gbm.score(x1_train_res, y1_train_res)
#recall_score(y1_train_res, gbm.predict(x1_train_res))
#precision_score(y1_train_res, gbm.predict(x1_train_res))

In [ ]:
x1_val_arr=np.array(x1_val)
y1_val_arr=np.array(y1_val)
gbm.score(x1_val_arr, y1_val_arr)

In [ ]:
recall_score(y1_val_arr, gbm.predict(x1_val_arr))
precision_score(y1_val_arr, gbm.predict(x1_val_arr))
f1_score(y1_val_arr, gbm.predict(x1_val_arr))

In [ ]:
#len(prediction_rf_model)

In [ ]:
#train_data.head()

In [ ]:
np.savetxt("/Users/s0c02nj/Desktop/WNS/model_oversampling.csv",prediction_rf_model)

In [ ]:
#cat_encoded.dtypes
X_data=np.array(cat_encoded)
y_data=np.array(train_data.is_promoted)

In [ ]:
#Sklearnmodel
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_validate

rf_model=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

#rf.fit(X_data, y_data)

In [ ]:
#Precision
cross_validate(rf_model, X_data, y_data, cv=10,return_train_score=False,scoring='precision')

In [ ]:
cross_validate(rf_model, X_data, y_data, cv=10,return_train_score=False,scoring='recall')

In [ ]:
#Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_data, y_data,test_size = .2,random_state=12)

In [ ]:
sm = SMOTE(random_state=12, ratio = 1)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=100, random_state=12)
clf_rf.fit(x_train_res, y_train_res)

In [ ]:
clf_rf.score(x_val, y_val)
recall_score(y_val, clf_rf.predict(x_val))
precision_score(y_val, clf_rf.predict(x_val))
f1_score(y_val, clf_rf.predict(x_val))